# Parsing WebSIS Student Lists

The file `generate.py` of the lecture code contains some functions for generating test data in various formats.  In class, we use student data created randomly by [a nifty web service](http://www.generatedata.com/).  We can ask for the data in the [JSON format](https://en.wikipedia.org/wiki/JSON), which Python conveniently accepts as code.  Then one of our functions outputs the data to a text file mimicking what WebSIS provides to instructors.

To generate the example input files yourself, run
      
      python3 generate.py

Check out `websis.txt`.  It's an interesting puzzle to work through how to process WebSIS data into the natural Python data structures.  Many of the puzzles we've looked at in this class seem hard even when we try to compute answers ourselves on a blackboard.  In contrast, processing a text file often seems trivial at first, and yet we encounter all sorts of fiddly details in trying to write code that does it properly.

First, let's build a simple generator to yield all characters in a text file that we've opened.

In [61]:
def characters(f):
    """For f an open file, yield all its characters in order."""
    
    while True:
        c = f.read(1)
        if not c:
            return
        yield c

Here's another handy helper function.

In [62]:
def allCharactersIn(checkForThese, inThis):
    """Check if all characters of one string are also in the other string."""
    
    for ch in inThis:
        if not ch in checkForThese:
            return False
    return True

The next key ingredient will be processing the header line that has a whole bunch of dashes on it, indicating the boundaries of the different fields in the rows that follow.

In [63]:
def dashSpans(s):
    """Given a string of dashes and nondash characters, return the beginning of each sequence of dashes.
    The output is a list of the numeric positions of those first characters, within the string."""
    
    alreadyInDash = False
    output = []

    for i in range(len(s)):
        if not alreadyInDash and s[i] == '-':
            output.append(i)
        alreadyInDash = (s[i] == '-')

    return output

In the WebSIS format, when a field value is shorter than the total length allocated to the field, extra spaces appear.  Let's write a function to remove the extras.

In [64]:
def removeTrailing(s):
    """Return a string that omits trailing spaces and newlines."""
    for i in reversed(range(len(s))):
        # The following check succeeds on the _last_ nonspace character in the string.
        if s[i] != ' ' and s[i] != '\n':
            return s[:i+1]
    return ""

Now we can process the column headers.

In [65]:
def columnHeaders(s, spans):
    """Given the dash spans of a string, extract the column headers from a
    string standing for the immediately previous line in the file."""
    
    lastSpan = None
    output = []

    for span in spans:
        if lastSpan:
            output.append(removeTrailing(s[lastSpan:span]))
        lastSpan = span

    output.append(removeTrailing(s[lastSpan:]))
    return output

Next, here's how we can process one row of data.

In [66]:
def removeLeading(s):
    """Return a string that omits leading spaces and newlines."""
    for i in range(len(s)):
        if s[i] != ' ' and s[i] != '\n':
            return s[i:]
    return ""

def trim(s):
    """Remove both leading and trailing spaces and newlines."""
    return removeLeading(removeTrailing(s))

def readRow(spans, headers, s):
    """Given information on field spans and headers, process a single line of text
    into a dictionary with one field per header."""
    output = {}

    for i in range(len(spans)):
        if i < len(spans)-1:
            value = s[spans[i]:spans[i+1]]
        else:
            value = s[spans[i]:]
        output[headers[i]] = trim(value)

    return output

Finally, we tie it all together.

In [67]:
def parseWebsis(filename):
    """Read a single WebSIS student-enrollment record from the named file."""
    with open(filename, "r") as f:
        last_line = ""

        # Step 1: loop until we find a line of all spaces and dashes.
        # This is the header that defines which fields exist.
        for line in f:
            if allCharactersIn(" -\n", line) and not allCharactersIn(" \n", line):
                spans = dashSpans(line)
                headers = columnHeaders(last_line, spans)
                break
            last_line = line

        # Step 2: loop reading all of the data rows.
        rows = []
        for line in f:
            rows.append(readRow(spans, headers, line))

        return rows

In [68]:
websis = parseWebsis('websis.txt')
websis

[{'MIT ID': '920073587',
  'Student Name': 'Martin, Odette Q.',
  'Course': '6 3',
  'Y': '3',
  'St': 'Reg',
  'Un': '12',
  'Grade': '',
  'Enrolled': '6.666',
  'Sec': '',
  'Email Address': 'amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk'},
 {'MIT ID': '911887284',
  'Student Name': 'Clarke, Cyrus G.',
  'Course': '6 3',
  'Y': '2',
  'St': 'Reg',
  'Un': '12',
  'Grade': '',
  'Enrolled': '6.666',
  'Sec': '',
  'Email Address': 'orci.quis@dignissim.ca'},
 {'MIT ID': '919043935',
  'Student Name': 'Wolf, Omar B.',
  'Course': '6 3',
  'Y': '2',
  'St': 'Reg',
  'Un': '12',
  'Grade': '',
  'Enrolled': '6.666',
  'Sec': '',
  'Email Address': 'dui@enim.ca'},
 {'MIT ID': '924379468',
  'Student Name': 'Wagner, Erin C.',
  'Course': '6 3',
  'Y': '4',
  'St': 'Reg',
  'Un': '12',
  'Grade': '',
  'Enrolled': '6.666',
  'Sec': '',
  'Email Address': 'eu@Craseget.com'},
 {'MIT ID': '911034657',
  'Student Name': 'Melendez, Ora J.',
  'Course': '6 3',
  'Y': '3',
  'St': 'Reg',
  'U

# Parsing XML Grades Data

Our next challenge is to read the class grades spreadsheet, which we're imagining is in [XML](https://en.wikipedia.org/wiki/XML), an extremely popular textual format for tree-structured data.  XML is so popular, in fact, that you can open XML files in Firefox or Chrome and get some basic help navigating the trees.  We generate random grades XML files for testing purposes, and we can view them in any text editor, as well as some web browsers.  See file `grades.xml`.

We use some ad-hoc code for general XML parsing.  (Actually, we only handle a simple subset of XML, sufficient for this example.)  The algorithm is moderately straightforward, depending at each point on a *tag stack*.  XML documents represent trees, and, at any point in parsing, we are processing some *tag*, or tree node.  The stack at that point stores all the ancestors of the current tag, in order.  As we finish parsing a node, we add that node to the children list of its parent, and we need the stack to find that parent.

In [69]:
def parseXml(filename):
    """Parse one XML tag (with children) from a file."""
    with open(filename, "r") as f:
        # We maintain a stack of tags that we are still working on.
        # Each is an object that contains a list of children.
        # We start with a dummy tag standing for the document root.
        stack = [{"tag": "root", "children": [], "text": ""}]

        # Now we loop until there is no more content to read.
        while True:
            # Step 1: Find the start of the open tag.
            for ch in characters(f):
                if ch == '<':
                    # Oo, an open tag!  Exit the loop and start processing the tag name.
                    break
                else:
                    # This character has no special interpretation.  Add it to the string for the parent tag.
                    stack[-1]["text"] += ch

            # Step 2: Extract the tag name.
            #after breaking, code will move here
            is_closer = False
            tag = ""
            for ch in characters(f):
                if ch == '/':
                    # Oh, a closing tag.  We'll need to pop from the stack afterward.
                    #start of the closing tag
                    is_closer = True
                #this may signify a opening or closing tag
                elif ch == '>':
                    # Found the end
                    break
                else:
                    # Found another character in the tag name.
                    tag += ch

            # Step 3: Manipulate the tag stack appropriately.
            if tag == "":
                # We seem to have reached the end of the file.
                break
            elif is_closer:
                # For a closing tag, pop it off, adding it to the children list of the top of the stack,
                # after removing leading and trailing spaces.
                this_node = stack.pop()
                this_node["text"] = trim(this_node["text"])
                stack[-1]["children"].append(this_node)
            else:
                # For an opening tag, add it to the stack.
                stack.append({"tag": tag, "children": [], "text": ""})

        # Now we should find the main tag as the sole child of the root node.
        return stack[0]["children"][0]

In [70]:
grades = parseXml("grades.xml")
grades

{'tag': 'grades',
 'children': [{'tag': 'psets',
   'children': [{'tag': 'pset',
     'children': [{'tag': 'number', 'children': [], 'text': '1'},
      {'tag': 'student',
       'children': [{'tag': 'email',
         'children': [],
         'text': 'amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk'},
        {'tag': 'grade', 'children': [], 'text': '93'}],
       'text': ''},
      {'tag': 'student',
       'children': [{'tag': 'email',
         'children': [],
         'text': 'orci.quis@dignissim.ca'},
        {'tag': 'grade', 'children': [], 'text': '54'}],
       'text': ''},
      {'tag': 'student',
       'children': [{'tag': 'email', 'children': [], 'text': 'dui@enim.ca'},
        {'tag': 'grade', 'children': [], 'text': '93'}],
       'text': ''},
      {'tag': 'student',
       'children': [{'tag': 'email',
         'children': [],
         'text': 'eu@Craseget.com'},
        {'tag': 'grade', 'children': [], 'text': '92'}],
       'text': ''},
      {'tag': 'student',
    

We postprocess the XML to generate more natural Python data structures, for instance using dictionaries in place of lists of child tags.

In [71]:
def assignmentFromXml(xml):
    """Convert one assignment (pset or quiz) to a nicer format."""
    
    number = [int(child["text"])
              for child in xml["children"]
              if child["tag"] == "number"][0]

    students = {email["text"]: int(grade["text"])
                for child in xml["children"]
                if child["tag"] == "student"
                for email in child["children"]
                if email["tag"] == "email"
                for grade in child["children"]
                if grade["tag"] == "grade"}

    return {"number": number, "students": students}

def categoryFromXml(xml, singular):
    """Convert one category (psets or quizzes) to a nicer format."""
    
    return {item["number"]: item["students"]
            for child in xml["children"]
            if child["tag"] == singular
            for item in [assignmentFromXml(child)]}

def gradesFromXml(xml):
    """Convert a whole grades database from XML to a nicer format."""
    
    psets = [child
             for child in xml["children"]
             if child["tag"] == "psets"][0]
    quizzes = [child
               for child in xml["children"]
               if child["tag"] == "quizzes"][0]

    return {"psets": categoryFromXml(psets, "pset"),
            "quizzes": categoryFromXml(quizzes, "quiz")}

Bringing it all together:

In [72]:
def parseGrades(filename):
    """Return the nice version of the grades database found in the file."""
    
    return gradesFromXml(parseXml(filename))

In [73]:
grades = parseGrades("grades.xml")
grades

{'psets': {1: {'amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk': 93,
   'orci.quis@dignissim.ca': 54,
   'dui@enim.ca': 93,
   'eu@Craseget.com': 92,
   'aliquet.vel@Vestibulum.org': 75,
   'at@massa.net': 78,
   'in@iaculis.org': 25,
   'porttitor.eros@ametdapibus.com': 96,
   'orci.tincidunt.adipiscing@lobortisrisusIn.org': 82,
   'Curabitur.egestas.nunc@erat.org': 73,
   'ante.ipsum@Phasellusat.edu': 77,
   'faucibus@elitNulla.edu': 86,
   'Nunc.ullamcorper@aceleifend.net': 85,
   'dolor.Quisque.tincidunt@quisarcuvel.com': 84,
   'turpis@milorem.ca': 88,
   'semper.auctor.Mauris@Maurismagna.com': 88,
   'blandit@augue.ca': 99,
   'tellus.imperdiet@necmetus.net': 63,
   'dui.quis@pretiumneque.com': 82,
   'tristique.aliquet.Phasellus@dictumeuplacerat.ca': 100,
   'sit@duiSuspendisse.edu': 100,
   'pede.Nunc.sed@justoProin.co.uk': 82,
   'eu.ultrices.sit@tempuseu.ca': 95,
   'Cras.pellentesque.Sed@ligulatortor.ca': 89,
   'ut.nulla@Aliquam.edu': 76,
   'sem.Nulla@lacuspedesagittis

After all that fuss implementing an XML parser, we could have just used one that comes with Python.  Here's an example showing how.

In [74]:
import xml.etree.ElementTree as ET

def assignmentFromXmlEasier(xml):
    number = int(xml.find("number").text)
    students = {child.find("email").text: int(child.find("grade").text)
                for child in xml.findall("student")}

    return {"number": number, "students": students}

def categoryFromXmlEasier(xml, singular):
        return {item["number"]: item["students"]
                for child in xml.findall(singular)
                for item in [assignmentFromXmlEasier(child)]}

def gradesFromXmlEasier(xml):
        return {"psets": categoryFromXmlEasier(xml.find("psets"), "pset"),
                "quizzes": categoryFromXmlEasier(xml.find("quizzes"), "quiz")}

def parseGradesEasier(filename):
        return gradesFromXmlEasier(ET.parse(filename).getroot())

In [75]:
parseGradesEasier('grades.xml')

{'psets': {1: {'amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk': 93,
   'orci.quis@dignissim.ca': 54,
   'dui@enim.ca': 93,
   'eu@Craseget.com': 92,
   'aliquet.vel@Vestibulum.org': 75,
   'at@massa.net': 78,
   'in@iaculis.org': 25,
   'porttitor.eros@ametdapibus.com': 96,
   'orci.tincidunt.adipiscing@lobortisrisusIn.org': 82,
   'Curabitur.egestas.nunc@erat.org': 73,
   'ante.ipsum@Phasellusat.edu': 77,
   'faucibus@elitNulla.edu': 86,
   'Nunc.ullamcorper@aceleifend.net': 85,
   'dolor.Quisque.tincidunt@quisarcuvel.com': 84,
   'turpis@milorem.ca': 88,
   'semper.auctor.Mauris@Maurismagna.com': 88,
   'blandit@augue.ca': 99,
   'tellus.imperdiet@necmetus.net': 63,
   'dui.quis@pretiumneque.com': 82,
   'tristique.aliquet.Phasellus@dictumeuplacerat.ca': 100,
   'sit@duiSuspendisse.edu': 100,
   'pede.Nunc.sed@justoProin.co.uk': 82,
   'eu.ultrices.sit@tempuseu.ca': 95,
   'Cras.pellentesque.Sed@ligulatortor.ca': 89,
   'ut.nulla@Aliquam.edu': 76,
   'sem.Nulla@lacuspedesagittis

# Outputting Online Grades Data

Now we've processed the two data sources, and it's time to create a file that OGS will accept.  We can take a look at [the OGS documentation](https://registrar.mit.edu/sites/default/files/2018-12/ogs_import_instructions.pdf) to learn about the format.  It's based on a common convention called [CSV](https://en.wikipedia.org/wiki/Comma-separated_values), for Comma-Separated Values.  The convention is probably clear almost immediately after looking at the example data, and it's also easy to process with code, either handrolled or from a popular library.  Pretty much any popular spreadsheet program can load and save CSV files.

It's not hard to write the code to output final grade information in our example.  The most serious gymnastics come from splitting a string with a student's name into first, middle, and last names.  First we write some manual code, coming back afterward to a shorter version using more of the Python standard library.

Annoyingly, WebSIS gives student names as combined strings, while Online Grades wants separate last and first names and middle initials!  Let's write a function to make a best-effort decomposition.

In [76]:
def splitName(name):
    # These variables will store positions in the string.
    comma = None
    firstSpace = None
    secondSpace = None

    for i in range(len(name)):
        if name[i] == ',' and not comma:
            comma = i
        elif name[i] == ' ' and not firstSpace:
            firstSpace = i
        elif name[i] == ' ':
            secondSpace = i

    return name[:comma], name[firstSpace+1:secondSpace], name[secondSpace+1:secondSpace+2]

In [77]:
splitName('Doe, John Q.')

('Doe', 'John', 'Q')

It turns out that there's a much simpler way to do this kind of string matching, using Python's standard library and an idea called _regular expressions_.

In [78]:
import re

def splitNameWithRegex(name):
    m = re.search("(.*), (.*) (.*)\\.", name)
    return m.group(1), m.group(2), m.group(3)

In [79]:
splitNameWithRegex('Doe, John Q.')

('Doe', 'John', 'Q')

The next important ingredient is the one that actually computes the letter grade!  We need to compute percentages and map them to letters.

In [80]:
def averageToLetter(n):
    if n >= 90:
        return "A"
    elif n >= 80:
        return "B"
    elif n >= 70:
        return "C"
    elif n >= 60:
        return "D"
    else:
        return "A+"

To keep this simple, we'll weight all psets/quizzes equally.

In [81]:
def studentAverage(email, grades):
    total = 0

    for pset in grades["psets"].values():
        if email in pset:
            total += pset[email]

    for quiz in grades["quizzes"].values():
        if email in quiz:
            total += quiz[email]

    return total / (len(grades["psets"]) + len(grades["quizzes"]))

def studentGrade(email, grades):
    return averageToLetter(studentAverage(email, grades))

In [82]:
studentAverage('aliquet.vel@Vestibulum.org', grades)

88.58333333333333

In [83]:
studentGrade('aliquet.vel@Vestibulum.org', grades)

'B'

Now the end of our long journey: outputting the file suitable for OGS import.

In [84]:
def outputGrades(filename, students, grades):
    with open(filename, "w") as f:
        f.write("Last Name,First Name,Middle,MIT ID,Subject #,Section #,Grade,Units,Comment\n")

        for student in students:
            last, first, middle = splitName(student["Student Name"])
            f.write(last + "," + first + "," + middle + "," \
                    + student["MIT ID"] + "," \
                    + student["Enrolled"] + "," \
                    + student["Sec"] + "," \
                    + studentGrade(student["Email Address"], grades) + "," \
                    + student["Un"] + "," \
                    + "\n")

In [85]:
outputGrades('ogs.csv', websis, grades)

# Reading Multiple Input Files

Sometimes data arrive as many files sitting in one directory.  In such cases, we need to be able to list all files in a directory, processing each one and combining the results somehow.  Consider the example of an alternative grades input format with one file per pset or quiz, found in generated subdirectory `grades`.  Each file is a CSV giving student e-mail addresses and numeric scores.  Our solution shows off more regular-expression operators.

In [86]:
import os

def parseCsvLine(line):
    """Parse one line of a CSV file, breaking it into a list of field values."""
    parts = []

    while True:
        m = re.search("([^,]*),(.*)", line)
        if not m:
            parts.append(line)
            return parts
        else:
            parts.append(m.group(1))
            line = m.group(2)

def readCsvFile(filename):
    """Read a whole CSV file that begins with a header line.
    Returns a list of dictionaries, with keys corresponding to header texts."""
    
    students = []

    with open(filename, "r") as f:
        headers = parseCsvLine(f.readline())

        for line in f:
            parts = parseCsvLine(line)
            student = {headers[i]: parts[i] for i in range(len(headers))}
            students.append(student)

        return students

def readMultiFile(dirname):
    """Read all CSV files in a directory, to produce a grades database
    of the kind we built above."""
    
    out = {"psets": {},
           "quizzes": {}}

    for filename in os.listdir(dirname):
        lines = readCsvFile(os.path.join(dirname, filename))
        mapping = {line["Email Address"]: int(line["Grade"]) for line in lines}

        m = re.search("pset([0-9]*)\\.csv", filename)
        if m:
            out["psets"][int(m.group(1))] = mapping
        else:
            m = re.search("quiz([0-9]*)\\.csv", filename)
            if m:
                out["quizzes"][int(m.group(1))] = mapping

    return out

In [87]:
readMultiFile('grades')

{'psets': {8: {'amet.ornare.lectus@Pellentesquehabitantmorbi.co.uk': 90,
   'orci.quis@dignissim.ca': 81,
   'dui@enim.ca': 90,
   'eu@Craseget.com': 100,
   'aliquet.vel@Vestibulum.org': 72,
   'at@massa.net': 93,
   'in@iaculis.org': 71,
   'porttitor.eros@ametdapibus.com': 85,
   'orci.tincidunt.adipiscing@lobortisrisusIn.org': 97,
   'Curabitur.egestas.nunc@erat.org': 97,
   'ante.ipsum@Phasellusat.edu': 97,
   'faucibus@elitNulla.edu': 90,
   'Nunc.ullamcorper@aceleifend.net': 93,
   'dolor.Quisque.tincidunt@quisarcuvel.com': 93,
   'turpis@milorem.ca': 95,
   'semper.auctor.Mauris@Maurismagna.com': 84,
   'blandit@augue.ca': 83,
   'tellus.imperdiet@necmetus.net': 100,
   'dui.quis@pretiumneque.com': 83,
   'tristique.aliquet.Phasellus@dictumeuplacerat.ca': 75,
   'sit@duiSuspendisse.edu': 81,
   'pede.Nunc.sed@justoProin.co.uk': 82,
   'eu.ultrices.sit@tempuseu.ca': 85,
   'Cras.pellentesque.Sed@ligulatortor.ca': 88,
   'ut.nulla@Aliquam.edu': 90,
   'sem.Nulla@lacuspedesagittis

# Writing Multiple Output Files

The last example generates output as multiple files in a fresh directory, giving each student a text file listing all grades.  This code relies on very few new tricks.

In [88]:
def makeStudentReports(dirname, students, grades):
    """Given values representing student data and grades, create the specified directory
    and populate it with one textual grade report per student."""
    os.mkdir(dirname)

    for student in students:
        with open(os.path.join(dirname, student["MIT ID"] + ".txt"), "w") as f:
            f.write("Dear " + student["Student Name"] + ",\n")
            f.write("\n")
            f.write("How are you enjoying Course " + student["Course"] + "?  Here are your grades in 6.666.\n")
            f.write("\n")

            f.write("Psets:\n")
            for pset, scores in grades["psets"].items():
                if student["Email Address"] in scores:
                    f.write("#" + str(pset) + ": " + str(scores[student["Email Address"]]) + "\n")
            f.write("\n")

            f.write("Quizzes:\n")
            for quiz, scores in grades["quizzes"].items():
                if student["Email Address"] in scores:
                    f.write("#" + str(quiz) + ": " + str(scores[student["Email Address"]]) + "\n")

In [89]:
makeStudentReports('reports', websis, grades)

# Summary

Our coordinated suite of examples illustrates a number of practical points about working with real-world datasets:
 1. The same data could arrive in many different formats.
 2. There are many different ways of remixing the same underlying data.
 3. The same results can be output in many different ways.

When we have to deal with a change in just one of these three aspects, it's very convenient to be able to reuse our code for the others, which suggests how important is to use standard data-structure formats internally.